# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import re
import pickle
import nltk
import sqlite3

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

In [2]:
# Download nltk packages
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bjpay\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bjpay\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bjpay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\bjpay\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
# load data from database
conn = sqlite3.connect('data/DisasterResponse.sqlite')
df = pd.read_sql('SELECT * FROM categorized_messages', conn)

df.head()

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,1,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
2,2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df['message']
y = df.drop(['index', 'id', 'message', 'original', 'genre'], axis=1)

In [5]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [6]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
print(X.shape)
print(y.shape)

(26216,)
(26216, 36)


### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    # Normalize text
    text = text.strip()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    text = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", '', text)

    # Fetch tokens
    tokens = []

    english_stopwords = stopwords.words("english")

    lemmatizer = WordNetLemmatizer()

    stemmer = PorterStemmer()

    for token in word_tokenize(text):
        if token not in english_stopwords:
            lemmatized = lemmatizer.lemmatize(token)

            stemmed = stemmer.stem(lemmatized)

            tokens.append(stemmed)

    return tokens

In [9]:
# Check output
print(tokenize(X[10]))
print(tokenize(X[34]))
print(tokenize(X[26215]))

['noth', 'eat', 'water', 'starv', 'thirsti']
['want', 'know', 'carrefour', 'need', 'help', 'starv', 'death']
['radic', 'shift', 'think', 'came', 'result', 'meet', 'recogn', 'hiv', 'aid', 'core', 'humanitarian', 'crisi', 'identifi', 'crisi', 'function', 'hiv', 'aid', 'pandem']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

# train classifier 
pipeline.fit(X_train, y_train) 

# predict on test data 
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
print(y_test.shape)
print(y_pred.shape)

(6554, 36)
(6554, 36)


In [17]:
y.columns.values

array(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers',
       'other_infrastructure', 'weather_related', 'floods', 'storm',
       'fire', 'earthquake', 'cold', 'other_weather', 'direct_report'],
      dtype=object)

In [18]:
# Model Evaluation metrics
print(classification_report(y_test, y_pred, target_names=y_test.columns))

                        precision    recall  f1-score   support

               related       0.84      0.96      0.89      4987
               request       0.83      0.48      0.61      1114
                 offer       0.00      0.00      0.00        28
           aid_related       0.77      0.69      0.73      2721
          medical_help       0.65      0.10      0.17       531
      medical_products       0.78      0.06      0.12       328
     search_and_rescue       0.58      0.04      0.08       163
              security       0.00      0.00      0.00       109
              military       0.67      0.03      0.06       237
           child_alone       0.00      0.00      0.00         0
                 water       0.87      0.39      0.54       424
                  food       0.86      0.57      0.69       741
               shelter       0.78      0.38      0.51       554
              clothing       0.74      0.14      0.24        99
                 money       1.00      

c:\projects\udacity\distasterresponsepipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\projects\udacity\distasterresponsepipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\projects\udacity\distasterresponsepipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline.get_params()

In [33]:
print(pipeline.get_params().keys())

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf_

In [38]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, max_df=0.8)),
    ('tfidf', TfidfTransformer(sublinear_tf=True)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))
])

parameters = {
    'clf__estimator__criterion': ['gini', 'entropy'],
    'clf__estimator__max_features': ['auto', 'sqrt', 'log2']
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=3, verbose=3)

cv.fit(X_train, y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV 1/3] END clf__estimator__criterion=gini, clf__estimator__max_features=auto;, score=0.264 total time= 1.4min
[CV 2/3] END clf__estimator__criterion=gini, clf__estimator__max_features=auto;, score=0.260 total time= 1.3min
[CV 3/3] END clf__estimator__criterion=gini, clf__estimator__max_features=auto;, score=0.269 total time= 1.3min
[CV 1/3] END clf__estimator__criterion=gini, clf__estimator__max_features=sqrt;, score=0.261 total time= 1.3min
[CV 2/3] END clf__estimator__criterion=gini, clf__estimator__max_features=sqrt;, score=0.260 total time= 1.3min
[CV 3/3] END clf__estimator__criterion=gini, clf__estimator__max_features=sqrt;, score=0.267 total time= 1.3min
[CV 1/3] END clf__estimator__criterion=gini, clf__estimator__max_features=log2;, score=0.239 total time= 1.4min
[CV 2/3] END clf__estimator__criterion=gini, clf__estimator__max_features=log2;, score=0.244 total time= 1.4min
[CV 3/3] END clf__estimator__criterion=gini,

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(max_df=0.8,
                                                        tokenizer=<function tokenize at 0x000001FA5BBD9620>)),
                                       ('tfidf',
                                        TfidfTransformer(sublinear_tf=True)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))]),
             param_grid={'clf__estimator__criterion': ['gini', 'entropy'],
                         'clf__estimator__max_features': ['auto', 'sqrt',
                                                          'log2']},
             verbose=3)

In [39]:
y_pred_cv = cv.predict(X_test)

In [40]:
cv.best_params_

{'clf__estimator__criterion': 'gini', 'clf__estimator__max_features': 'auto'}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [41]:
# CV Model Evaluation metrics
print(classification_report(y_test, y_pred_cv, target_names=y_test.columns))

                        precision    recall  f1-score   support

               related       0.84      0.96      0.89      4987
               request       0.83      0.49      0.62      1114
                 offer       0.00      0.00      0.00        28
           aid_related       0.76      0.69      0.72      2721
          medical_help       0.54      0.06      0.11       531
      medical_products       0.75      0.06      0.12       328
     search_and_rescue       0.54      0.04      0.08       163
              security       0.00      0.00      0.00       109
              military       0.70      0.03      0.06       237
           child_alone       0.00      0.00      0.00         0
                 water       0.90      0.37      0.52       424
                  food       0.86      0.59      0.70       741
               shelter       0.79      0.36      0.49       554
              clothing       0.79      0.19      0.31        99
                 money       0.75      

c:\projects\udacity\distasterresponsepipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\projects\udacity\distasterresponsepipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\projects\udacity\distasterresponsepipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [42]:
filename = 'data/model.sav'

pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.